In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

### Init Values

In [33]:
import keras

train_percentaje = 1
validation_percentaje = 0
test_percentaje = 0
epochs_ = 1000

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

### Select data for each model

#### Global get_data

In [34]:
import numpy as np
import os

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    raise Exception(f"Not found the {genre}")

def get_data_from_path(data_path,genres, decoder):
    data = {'in': [], 'out': []}
    for genre in genres:
        files = os.listdir(data_path + genre)
        for filename in files:
            filepath = data_path + genre + '/' + filename
            data['in'].append(decoder(filepath))
            data['out'].append(index_genre(genre, genres))

    data = {'in': np.array(data['in']),'out': np.array(data['out'])}

    return data

#### Get data for MFCC model

In [35]:
import cv2
import os

size_images = (192, 256)

def read_image(filepath):
    global size_images
    image = cv2.imread(filepath)
    return cv2.resize(image, size_images)

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

data_path_train = 'path_train'
data_path_test = 'path_test'
data_path_validation = 'path_validation'

def get_data_train_mfcc():
    return get_data_from_path(data_path_train, genres, read_image)
def get_data_test_mfcc():
    return get_data_from_path(data_path_test, genres, read_image)
def get_data_validation_mfcc():
    return get_data_from_path(data_path_validation, genres, read_image)

mfcc_data = {} 
mfcc_data['data_training'] = get_data_train_mfcc()
mfcc_data['data_testing']= get_data_test_mfcc()
mfcc_data['data_validation']= get_data_validation_mfcc()

#### Get data for wavelet model

In [ ]:
!pip install dtcwt

In [37]:
import librosa
from collections import Counter
import scipy
import dtcwt


trans = dtcwt.Transform1d(biort='antonini', qshift='qshift_d')

def calculate_entropy(list_values):
	counter_values = Counter(list_values).most_common()
	probabilities = [elem[1]/len(list_values) for elem in counter_values]
	entropy=scipy.stats.entropy(probabilities)
	return entropy

def calculate_statistics(list_values):
	n5 = np.nanpercentile(list_values, 5)
	n25 = np.nanpercentile(list_values, 25)
	n75 = np.nanpercentile(list_values, 75)
	n95 = np.nanpercentile(list_values, 95)
	median = np.nanpercentile(list_values, 50)
	mean = np.nanmean(list_values)
	std = np.nanstd(list_values)
	var = np.nanvar(list_values)
	rms = np.nanmean(np.sqrt(list_values**2))
	return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
	zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
	no_zero_crossings = len(zero_crossing_indices)
	mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
	no_mean_crossings = len(mean_crossing_indices)
	return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):
	entropy = calculate_entropy(list_values)
	crossings = calculate_crossings(list_values)
	statistics = calculate_statistics(list_values)
	return [entropy] + crossings + statistics

def extract_dtcwt(file_path:str):
    d, fs = librosa.load(file_path)
    forw = trans.forward(d, nlevels=17)
    features = []
    for coeff in forw.highpasses:
        temp = (np.abs(coeff.squeeze()))
        features += get_features(temp)

    features += get_features(forw.lowpass.squeeze())
    return features

In [ ]:
data_path_train = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'
data_path_test = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'
data_path_validation = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'


def get_data_train_wavelet():
    return get_data_from_path(data_path_train, genres, extract_dtcwt)
def get_data_test_wavelet():
    return get_data_from_path(data_path_test, genres, extract_dtcwt)
def get_data_validation_wavelet():
    return get_data_from_path(data_path_test, genres, extract_dtcwt)

wavalet_data = {}

wavalet_data['data_training'] = get_data_train_wavelet()
wavalet_data['data_testing']= get_data_test_wavelet()
wavalet_data['data_validation']= get_data_validation_wavelet()

#### Get Conv1D data

In [ ]:
data_path = '/content/drive/MyDrive/GTZAN/Ensemble_Data/audios/models/'

# CARGAR UN ARCHIVO DE AUDIO
def load_audio_file(file_path):
    input_length = 660000  # This is 15 seconds with 44100 sample rate
    data = librosa.core.load(file_path, sr=22050)[0]  # We use librosa to load audio file with sample rate 22050
    if len(data) > input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
    return data

def get_data_conv1D(training_percentaje=0.8, validation_percentaje=0.1, test_percentaje=0.1):
    return get_data(data_path, genres, load_audio_file,training_percentaje,validation_percentaje,test_percentaje)

conv1D_data = get_data_conv1D()

conv1D_data['data_training']['in'] = conv1D_data['data_training']['in']
conv1D_data['data_validation']['in'] = conv1D_data['data_validation']['in']
conv1D_data['data_testing']['in'] = conv1D_data['data_testing']['in']


#### Fill all datas

In [38]:
data = {
    'mfcc': mfcc_data,
    'wavalet': wavalet_data
}

### Import model.h5 for each model

In [40]:
import pickle


with open('/content/drive/MyDrive/Ensemble/dtcwt_rf.bin', 'rb') as archivo:
    dtcwt_rf_model = pickle.load(archivo)

mfcc_model = keras.models.load_model('/content/drive/MyDrive/Ensemble/mfcc_model.h5')
models = {
    'mfcc': mfcc_model,
    'wavalet':dtcwt_rf_model
}

### Train and Test (Ensemble Model)


#### Train

In [41]:
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense

classes = len(genres)

def train():
    train_data = []
    models_predict = []
    models_predict_validation = []

    for key,model_ in models.items():
        models_predict.append(model_.predict(data[key]['data_training']['in']))
        models_predict_validation.append(model_.predict(data[key]['data_validation']['in']))

    #Cambiar filas por columnas para tener n datos de tamanno len(modelos), que cada uno seria un array con la salida de cada modelo evaluado en los datos
    stacked_predictions = np.column_stack(models_predict)
    stacked_predictions_val = np.column_stack(models_predict_validation)

    model = Sequential()
    model.add(Dense(100, input_dim=len(stacked_predictions[0]), activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(classes, activation='softmax'))

    model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'])

    checkpoint = keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/Ensemble/best_weights_ensemble.h5',
        save_weights_only=True,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max')

    #se le pasa cualquier salida, ya que todas tendran igual tamanno e iguales
    model.fit(
        stacked_predictions, mfcc_data['data_training']['out'],
        validation_data=(stacked_predictions_val, mfcc_data['data_validation']['out']),
        epochs=epochs_,
        callbacks=[checkpoint]
        )

    model.load_weights('/content/drive/MyDrive/Ensemble/best_weights_ensemble.h5')
    model.save('/content/drive/MyDrive/Ensemble/ensemble_model.h5')
    return model

model = train()

4/4 [==============================] - 8s 1s/step
Epoch 1/1000
7/7 [==============================] - 3s 321ms/step - loss: 2.3090 - accuracy: 0.1508 - val_loss: 2.1972 - val_accuracy: 0.1583
Epoch 2/1000
7/7 [==============================] - 1s 190ms/step - loss: 2.1502 - accuracy: 0.3518 - val_loss: 2.0870 - val_accuracy: 0.5000
Epoch 3/1000
7/7 [==============================] - 1s 187ms/step - loss: 2.0474 - accuracy: 0.5879 - val_loss: 2.0038 - val_accuracy: 0.6417
Epoch 4/1000
7/7 [==============================] - 0s 10ms/step - loss: 1.9514 - accuracy: 0.6834 - val_loss: 1.9185 - val_accuracy: 0.6083
Epoch 5/1000
7/7 [==============================] - 0s 8ms/step - loss: 1.8351 - accuracy: 0.6633 - val_loss: 1.8087 - val_accuracy: 0.6167
Epoch 6/1000
7/7 [==============================] - 1s 172ms/step - loss: 1.6882 - accuracy: 0.7940 - val_loss: 1.6546 - val_accuracy: 0.7583
Epoch 7/1000
7/7 [==============================] - 1s 221ms/step - loss: 1.5256 - accuracy: 0.8392 -

#### Test

In [42]:
def test():
    models_predict_test = []
    for key,model_ in models.items():
        models_predict_test.append(model_.predict(data[key]['data_testing']['in']))

    input = np.column_stack(models_predict_test)
    output = mfcc_data['data_testing']['out']

    score = model.evaluate(input, output, verbose=0)
    print('testing accuracy: ' + str(score[1]))

test()

4/4 [==============================] - 5s 1s/step
testing accuracy: 0.924369752407074
